<a href="https://colab.research.google.com/github/BenjaminEngel919/Prediction-of-Product-Sales/blob/main/Benjamin_E's_Project_1_Part_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Benjamin Engel

##For Part 5, you will go back to your original, uncleaned, sales prediction dataset with the goal of preventing data leakage.

  *You should load a fresh version of the original data set here using pd.read_csv() and start your cleaning process over to ensure there is no data leakage!*

  1. Before splitting your data, you can drop duplicates and fix inconsistencies in categorical data.* (*There is a way to do this after the split, but for this project, you may perform this step before the split)
  2. Identify the features (X) and target (y): Assign the "Item_Outlet_Sales" column as your target and the rest of the relevant variables as your features matrix.
  3. Perform a train test split
  4. Create a preprocessing object to prepare the dataset for Machine Learning
  5. Make sure your imputation of missing values occurs after the train test split using SimpleImputer.

Commit your work to GitHub.

In [1]:
## Import and mount drive from Google Colab
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# The Usual Imports
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import make_column_transformer, make_column_selector
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn import set_config
set_config(display='diagram')

In [3]:
# Import the data
path = '/sales_predictions_2023.csv'
sales_predictions = pd.read_csv(path)
sales_predictions.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052


In [4]:
sales_predictions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8523 entries, 0 to 8522
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            8523 non-null   object 
 1   Item_Weight                7060 non-null   float64
 2   Item_Fat_Content           8523 non-null   object 
 3   Item_Visibility            8523 non-null   float64
 4   Item_Type                  8523 non-null   object 
 5   Item_MRP                   8523 non-null   float64
 6   Outlet_Identifier          8523 non-null   object 
 7   Outlet_Establishment_Year  8523 non-null   int64  
 8   Outlet_Size                6113 non-null   object 
 9   Outlet_Location_Type       8523 non-null   object 
 10  Outlet_Type                8523 non-null   object 
 11  Item_Outlet_Sales          8523 non-null   float64
dtypes: float64(4), int64(1), object(7)
memory usage: 799.2+ KB


##**Identify each feature**

- Pass through = none
- Ordinal features = None
- Numeric features = ['Item_Weight', 'Item_Visibility', 'Item_MRP]
- Nominal features = ['Item_Identifier', 'Item_Fat_Content', 'Item_Type', 'Outlet_Identifier', 'Outlet_Location_Type', 'Outlet_Size', 'Outlet_Type']
- Date/time features = ['Outlet_Establishment_Year']


##1a. Drop Duplicates

In [5]:
##3Any duplicates? If yes, drop them.
print(f'There are {sales_predictions.duplicated().sum()} duplicate rows.')

There are 0 duplicate rows.


#1b. Fix Inconsistencies in Categorical Data

In [6]:
print(sales_predictions.isna().sum())

Item_Identifier                 0
Item_Weight                  1463
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                  2410
Outlet_Location_Type            0
Outlet_Type                     0
Item_Outlet_Sales               0
dtype: int64


In [7]:
## Define the median value of the column
median_item_weight = sales_predictions['Item_Weight'].median()

In [8]:
## Replace the missing values with the column's 'median' value
sales_predictions['Item_Weight'].fillna(value = median_item_weight, inplace = True)

In [9]:
sales_predictions.fillna(value = 'Unknown', inplace=True)

In [10]:
##6 Confirm there are no missing values and address them!
print(sales_predictions.isna().sum())

Item_Identifier              0
Item_Weight                  0
Item_Fat_Content             0
Item_Visibility              0
Item_Type                    0
Item_MRP                     0
Outlet_Identifier            0
Outlet_Establishment_Year    0
Outlet_Size                  0
Outlet_Location_Type         0
Outlet_Type                  0
Item_Outlet_Sales            0
dtype: int64


In [11]:
data_types = sales_predictions.dtypes
str_cols = data_types[ data_types=='object'].index
str_cols

Index(['Item_Identifier', 'Item_Fat_Content', 'Item_Type', 'Outlet_Identifier',
       'Outlet_Size', 'Outlet_Location_Type', 'Outlet_Type'],
      dtype='object')

In [12]:
for col in str_cols:
  print(f'- {col}:')
  print(sales_predictions[col].value_counts(dropna=False))
  print("\n\n")

- Item_Identifier:
FDW13    10
FDG33    10
NCY18     9
FDD38     9
DRE49     9
         ..
FDY43     1
FDQ60     1
FDO33     1
DRF48     1
FDC23     1
Name: Item_Identifier, Length: 1559, dtype: int64



- Item_Fat_Content:
Low Fat    5089
Regular    2889
LF          316
reg         117
low fat     112
Name: Item_Fat_Content, dtype: int64



- Item_Type:
Fruits and Vegetables    1232
Snack Foods              1200
Household                 910
Frozen Foods              856
Dairy                     682
Canned                    649
Baking Goods              648
Health and Hygiene        520
Soft Drinks               445
Meat                      425
Breads                    251
Hard Drinks               214
Others                    169
Starchy Foods             148
Breakfast                 110
Seafood                    64
Name: Item_Type, dtype: int64



- Outlet_Identifier:
OUT027    935
OUT013    932
OUT049    930
OUT046    930
OUT035    930
OUT045    929
OUT018    928
OUT017    9

In [13]:
# Display the unique values from the column
print(sales_predictions['Item_Fat_Content'].unique())

['Low Fat' 'Regular' 'low fat' 'LF' 'reg']


In [14]:
##Using a Dictionary to replace value
sales_predictions.replace({'LF': 'Low Fat', 'reg': 'Regular', 'low fat': 'Low Fat'}, inplace = True)

In [15]:
## Displaying the new value totals
sales_predictions['Item_Fat_Content'].value_counts()

Low Fat    5517
Regular    3006
Name: Item_Fat_Content, dtype: int64

In [16]:
# 'Outlet_Establishment_Year' is a date/time and will be included with categorical features
sales_predictions['Outlet_Establishment_Year'] = sales_predictions['Outlet_Establishment_Year'].astype("object")

##2. Define features (X) and target (y)

In [17]:
#Defining my Target variable which is y and the remaining features X
X = sales_predictions.drop(columns = ['Item_Identifier','Item_Outlet_Sales'])
y = sales_predictions['Item_Outlet_Sales']

In [19]:
sales_predictions.dtypes

Item_Identifier               object
Item_Weight                  float64
Item_Fat_Content              object
Item_Visibility              float64
Item_Type                     object
Item_MRP                     float64
Outlet_Identifier             object
Outlet_Establishment_Year     object
Outlet_Size                   object
Outlet_Location_Type          object
Outlet_Type                   object
Item_Outlet_Sales            float64
dtype: object

#3. Train Test Split

In [20]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42)

In [21]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6392 entries, 4776 to 7270
Data columns (total 10 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Weight                6392 non-null   float64
 1   Item_Fat_Content           6392 non-null   object 
 2   Item_Visibility            6392 non-null   float64
 3   Item_Type                  6392 non-null   object 
 4   Item_MRP                   6392 non-null   float64
 5   Outlet_Identifier          6392 non-null   object 
 6   Outlet_Establishment_Year  6392 non-null   object 
 7   Outlet_Size                6392 non-null   object 
 8   Outlet_Location_Type       6392 non-null   object 
 9   Outlet_Type                6392 non-null   object 
dtypes: float64(3), object(7)
memory usage: 549.3+ KB


In [22]:
X_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2131 entries, 7503 to 1784
Data columns (total 10 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Weight                2131 non-null   float64
 1   Item_Fat_Content           2131 non-null   object 
 2   Item_Visibility            2131 non-null   float64
 3   Item_Type                  2131 non-null   object 
 4   Item_MRP                   2131 non-null   float64
 5   Outlet_Identifier          2131 non-null   object 
 6   Outlet_Establishment_Year  2131 non-null   object 
 7   Outlet_Size                2131 non-null   object 
 8   Outlet_Location_Type       2131 non-null   object 
 9   Outlet_Type                2131 non-null   object 
dtypes: float64(3), object(7)
memory usage: 183.1+ KB


#4. Instantiate Column Selectors

In [23]:
#Column Selectors
cat_selector = make_column_selector(dtype_include='object')
num_selector = make_column_selector(dtype_include='number')

In [24]:
cat_selector(X_train)

['Item_Fat_Content',
 'Item_Type',
 'Outlet_Identifier',
 'Outlet_Establishment_Year',
 'Outlet_Size',
 'Outlet_Location_Type',
 'Outlet_Type']

In [25]:
num_selector(X_train)

['Item_Weight', 'Item_Visibility', 'Item_MRP']

#5a. Instantiate Transformers

In [26]:
#Imputers
freq_imputer = SimpleImputer(strategy='most_frequent')
median_imputer = SimpleImputer(strategy='median')
#Scaler
scaler = StandardScaler()
#One-hot encoder
ohe = OneHotEncoder(handle_unknown='ignore', sparse=False)

#5b. Instantiate Pipelines

In [27]:
#Numeric pipeline
numeric_pipe = make_pipeline(median_imputer, scaler)
numeric_pipe

Pipeline(steps=[('simpleimputer', SimpleImputer(strategy='median')),
                ('standardscaler', StandardScaler())])

In [28]:
#Categorical pipeline
categorical_pipe = make_pipeline(freq_imputer, ohe)
categorical_pipe

Pipeline(steps=[('simpleimputer', SimpleImputer(strategy='most_frequent')),
                ('onehotencoder',
                 OneHotEncoder(handle_unknown='ignore', sparse=False))])

#5c. Instantiate Column Transformer

In [29]:
#Tuples for Column Transformer
number_tuple = (numeric_pipe, num_selector)
category_tuple = (categorical_pipe, cat_selector)

#ColumnTransformer
preprocessor = make_column_transformer(number_tuple, category_tuple, remainder='passthrough')
preprocessor

ColumnTransformer(remainder='passthrough',
                  transformers=[('pipeline-1',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(strategy='median')),
                                                 ('standardscaler',
                                                  StandardScaler())]),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x7efbc792af50>),
                                ('pipeline-2',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('onehotencoder',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False))]),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x7efbf9f6d720>)])

#**Fit and Transform Data**

In [30]:
#Fit on Train data
preprocessor.fit(X_train)

ColumnTransformer(remainder='passthrough',
                  transformers=[('pipeline-1',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(strategy='median')),
                                                 ('standardscaler',
                                                  StandardScaler())]),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x7efbc792af50>),
                                ('pipeline-2',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('onehotencoder',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False))]),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x7efbf9f6d720>)])

In [31]:
preprocessor.fit(X_train).get_feature_names_out()

array(['pipeline-1__Item_Weight', 'pipeline-1__Item_Visibility',
       'pipeline-1__Item_MRP', 'pipeline-2__Item_Fat_Content_Low Fat',
       'pipeline-2__Item_Fat_Content_Regular',
       'pipeline-2__Item_Type_Baking Goods',
       'pipeline-2__Item_Type_Breads', 'pipeline-2__Item_Type_Breakfast',
       'pipeline-2__Item_Type_Canned', 'pipeline-2__Item_Type_Dairy',
       'pipeline-2__Item_Type_Frozen Foods',
       'pipeline-2__Item_Type_Fruits and Vegetables',
       'pipeline-2__Item_Type_Hard Drinks',
       'pipeline-2__Item_Type_Health and Hygiene',
       'pipeline-2__Item_Type_Household', 'pipeline-2__Item_Type_Meat',
       'pipeline-2__Item_Type_Others', 'pipeline-2__Item_Type_Seafood',
       'pipeline-2__Item_Type_Snack Foods',
       'pipeline-2__Item_Type_Soft Drinks',
       'pipeline-2__Item_Type_Starchy Foods',
       'pipeline-2__Outlet_Identifier_OUT010',
       'pipeline-2__Outlet_Identifier_OUT013',
       'pipeline-2__Outlet_Identifier_OUT017',
       'pipelin

In [32]:
# Transform Train and Test
X_train_processed = preprocessor.transform(X_train)
X_test_processed = preprocessor.transform(X_test)

In [33]:
# Display the datatype of X_train_scaled
type(X_train_processed)

numpy.ndarray

In [34]:
X_train_processed

array([[ 0.82944552, -0.71277507,  1.82810922, ...,  0.        ,
         1.        ,  0.        ],
       [ 0.5686337 , -1.29105225,  0.60336888, ...,  0.        ,
         1.        ,  0.        ],
       [-0.11896108,  1.81331864,  0.24454056, ...,  1.        ,
         0.        ,  0.        ],
       ...,
       [ 1.12582258, -0.92052713,  1.52302674, ...,  1.        ,
         0.        ,  0.        ],
       [ 1.77785212, -0.2277552 , -0.38377708, ...,  1.        ,
         0.        ,  0.        ],
       [ 0.82944552, -0.95867683, -0.73836105, ...,  1.        ,
         0.        ,  0.        ]])

#**Convert Preprocessed Array to Dataframes**

In [35]:
# Create dataframes from the processed arrays
# Default X_train and X_test
cols = preprocessor.fit(X_train).get_feature_names_out()

X_train_processed_df = pd.DataFrame(X_train_processed, 
                          columns = cols,
                          index = X_train.index)

X_test_processed_df = pd.DataFrame(X_test_processed, 
                         columns = cols,
                         index = X_test.index)

#**Inspect the Result**

In [36]:
#Check for missing values and that data is scaled and one-hot encoded#print(np.isnan(X_train_processed).sum().sum(), 'missing values in training data')
print(np.isnan(X_test_processed).sum().sum(), 'Missing Values in Testing Data.')
print('\n')
print('All Data in X_train_processed are', X_train_processed.dtype)
print('All Data in X_test_processed are', X_test_processed.dtype)
print('\n')
print('Shape of Data is', X_train_processed.shape)
print('\n')
#Check the data types
print(f'\nThe datatypes are {X_train_processed_df.dtypes}')
X_train_processed

0 Missing Values in Testing Data.


All Data in X_train_processed are float64
All Data in X_test_processed are float64


Shape of Data is (6392, 51)



The datatypes are pipeline-1__Item_Weight                        float64
pipeline-1__Item_Visibility                    float64
pipeline-1__Item_MRP                           float64
pipeline-2__Item_Fat_Content_Low Fat           float64
pipeline-2__Item_Fat_Content_Regular           float64
pipeline-2__Item_Type_Baking Goods             float64
pipeline-2__Item_Type_Breads                   float64
pipeline-2__Item_Type_Breakfast                float64
pipeline-2__Item_Type_Canned                   float64
pipeline-2__Item_Type_Dairy                    float64
pipeline-2__Item_Type_Frozen Foods             float64
pipeline-2__Item_Type_Fruits and Vegetables    float64
pipeline-2__Item_Type_Hard Drinks              float64
pipeline-2__Item_Type_Health and Hygiene       float64
pipeline-2__Item_Type_Household                float64
pipel

array([[ 0.82944552, -0.71277507,  1.82810922, ...,  0.        ,
         1.        ,  0.        ],
       [ 0.5686337 , -1.29105225,  0.60336888, ...,  0.        ,
         1.        ,  0.        ],
       [-0.11896108,  1.81331864,  0.24454056, ...,  1.        ,
         0.        ,  0.        ],
       ...,
       [ 1.12582258, -0.92052713,  1.52302674, ...,  1.        ,
         0.        ,  0.        ],
       [ 1.77785212, -0.2277552 , -0.38377708, ...,  1.        ,
         0.        ,  0.        ],
       [ 0.82944552, -0.95867683, -0.73836105, ...,  1.        ,
         0.        ,  0.        ]])

In [37]:
#View the transformations
X_train_processed_df.describe().round(2)

,pipeline-1__Item_Weight,pipeline-1__Item_Visibility,pipeline-1__Item_MRP,pipeline-2__Item_Fat_Content_Low Fat,pipeline-2__Item_Fat_Content_Regular,pipeline-2__Item_Type_Baking Goods,pipeline-2__Item_Type_Breads,pipeline-2__Item_Type_Breakfast,pipeline-2__Item_Type_Canned,pipeline-2__Item_Type_Dairy,...,pipeline-2__Outlet_Size_Medium,pipeline-2__Outlet_Size_Small,pipeline-2__Outlet_Size_Unknown,pipeline-2__Outlet_Location_Type_Tier 1,pipeline-2__Outlet_Location_Type_Tier 2,pipeline-2__Outlet_Location_Type_Tier 3,pipeline-2__Outlet_Type_Grocery Store,pipeline-2__Outlet_Type_Supermarket Type1,pipeline-2__Outlet_Type_Supermarket Type2,pipeline-2__Outlet_Type_Supermarket Type3
count,6392.00,6392.00,6392.00,6392.00,6392.00,6392.00,6392.00,6392.00,6392.00,6392.00,...,6392.00,6392.00,6392.00,6392.00,6392.00,6392.00,6392.00,6392.00,6392.00,6392.00
mean,-0.00,-0.00,0.00,0.65,0.35,0.07,0.03,0.01,0.08,0.08,...,0.33,0.28,0.28,0.27,0.33,0.40,0.12,0.65,0.11,0.11
std,1.00,1.00,1.00,0.48,0.48,0.26,0.16,0.11,0.26,0.27,...,0.47,0.45,0.45,0.45,0.47,0.49,0.33,0.48,0.31,0.32
min,-1.97,-1.29,-1.77,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
25%,-0.79,-0.76,-0.76,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
50%,-0.06,-0.23,0.03,1.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00
75%,0.77,0.56,0.72,1.00,1.00,0.00,0.00,0.00,0.00,0.00,...,1.00,1.00,1.00,1.00,1.00,1.00,0.00,1.00,0.00,0.00
max,2.01,5.13,1.99,1.00,1.00,1.00,1.00,1.00,1.00,1.00,...,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00
